# Grid Search 실습

### 데이터셋 다운로드

In [2]:
! wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z' -O pima-indians-diabetes.csv #https://drive.google.com/file/d/1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z/view?usp=sharing

--2021-06-01 08:06:57--  https://docs.google.com/uc?export=download&id=1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z
Resolving docs.google.com (docs.google.com)... 142.250.101.139, 142.250.101.138, 142.250.101.102, ...
Connecting to docs.google.com (docs.google.com)|142.250.101.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-1s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sr1h2kg72ccel6n3bqs0vq2l0hphi5km/1622534775000/04591858293269658552/*/1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z?e=download [following]
--2021-06-01 08:06:57--  https://doc-0o-1s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sr1h2kg72ccel6n3bqs0vq2l0hphi5km/1622534775000/04591858293269658552/*/1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z?e=download
Resolving doc-0o-1s-docs.googleusercontent.com (doc-0o-1s-docs.googleusercontent.com)... 142.250.101.132, 2607:f8b0:4023:c06::84
Connecting to doc-0o-1s-docs.googleusercontent.com (doc-0o

## batch size와 epoch 최적화

In [3]:
#@title
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.720052 using {'batch_size': 10, 'epochs': 100}
0.589844 (0.016877) with: {'batch_size': 10, 'epochs': 10}
0.653646 (0.022402) with: {'batch_size': 10, 'epochs': 50}
0.720052 (0.036690) with: {'batch_size': 10, 'epochs': 100}
0.578125 (0.030425) with: {'batch_size': 20, 'epochs': 10}
0.658854 (0.057880) with: {'batch_size': 20, 'epochs': 50}
0.710938 (0.011500) with: {'batch_size': 20, 'epochs': 100}
0.492188 (0.026107) with: {'batch_size': 40, 'epochs': 10}
0.649740 (0.019488) with: {'batch_size': 40, 'epochs': 50}
0.679688 (0.020915) with: {'batch_size': 40, 'epochs': 100}
0.604167 (0.013279) with: {'batch_size': 60, 'epochs': 10}
0.609375 (0.028348) with: {'batch_size': 60, 'epochs': 50}
0.647135 (0.028764) with: {'batch_size': 60, 'epochs': 100}
0.496094 (0.089304) with: {'batch_size': 80, 'epochs': 10}
0.613281 (0.024910) with: {'batch_size': 80, 'epochs': 50}
0.710938 (0.016877) with: {'batch_size': 80, 'epochs': 100}
0.585938 (0.088388) with: {'batch_size': 100, 'epochs':

## 학습률과 모멘텀 최적화

In [4]:
#@title
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
import time
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
	# create model
	modelx = Sequential()
	modelx.add(Dense(12, input_dim=8, activation='relu'))
	modelx.add(Dense(1, activation='sigmoid'))
	# Compile model
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	modelx.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return modelx
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
modelx = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=modelx, param_grid=param_grid, n_jobs=-1, cv=3)

# 트레이닝 시작
start = time.time()
print("트레이닝 시작")
grid_result = grid.fit(X, Y)
end = time.time()
print(end - start, ' 초 결렸습니다')    

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

트레이닝 시작


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


410.46758699417114  초 결렸습니다
Best: 0.694010 using {'learn_rate': 0.001, 'momentum': 0.6}
0.683594 (0.009568) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.669271 (0.011201) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.669271 (0.004872) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.694010 (0.029635) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.691406 (0.014616) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.651042 (0.027866) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.598958 (0.056727) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.653646 (0.036690) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.666667 (0.009207) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.649740 (0.026557) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.651042 (0.024774) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.652344 (0.025315) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.651042 (0.

In [5]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
import time
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
	# create model
	modelx = Sequential()
	modelx.add(Dense(12, input_dim=8, activation='relu'))
	modelx.add(Dense(1, activation='sigmoid'))
	# Compile model
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	modelx.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return modelx
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
modelx = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
# learn_rate와 momentum 두 개의 하이퍼파라미터의 그리드를 코드로 정의하십시오. 
# learn_rate는 0.001, 0.01. 0.1, 0.2, 0.3로 하고 momentum은 0.0, 0.2, 0.4, 0.6, 0.8, 0.9로 하십시오.
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=modelx, param_grid=param_grid, n_jobs=-1, cv=3)

# 트레이닝 시작
start = time.time()
print("트레이닝 시작")
grid_result = grid.fit(X, Y)
end = time.time()
print(end - start, ' 초 결렸습니다') 

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


트레이닝 시작


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


426.25091767311096  초 결렸습니다
Best: 0.695312 using {'learn_rate': 0.001, 'momentum': 0.0}
0.695312 (0.030425) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.684896 (0.010253) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.665365 (0.031466) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.692708 (0.015073) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.651042 (0.040133) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.678385 (0.028764) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.640625 (0.041463) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.654948 (0.028940) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.630208 (0.041010) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.654948 (0.026748) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.649740 (0.026557) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.651042 (0.024774) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.649740 (0.024360) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.651042 (0.

In [8]:
from keras.layers import Dropout 
layer = Dropout(rate=0.2, input_shape=(2,))

In [9]:
from keras.layers import LeakyReLU
layer = LeakyReLU(alpha=0.1)